In [ ]:
import feedparser

# URL del feed de Yahoo Finance para NVIDIA
url = "https://feeds.finance.yahoo.com/rss/2.0/headline?s=NVDA&region=US&lang=en-US"

# Leer el feed
feed = feedparser.parse(url)

# Mostrar los títulos
for entry in feed.entries:
    print("Título:", entry.title)
    print("Enlace:", entry.link)
    print("Resumen:", entry.summary)
    print("------")

feed.keys()
feed.entries[0]

Título: BT CEO hails telecom networks as 'UK's real digital backbone'
Enlace: https://uk.finance.yahoo.com/news/bt-ceo-london-tech-week-uk-infrastructure-050059140.html?.tsrc=rss
Resumen: Allison Kirkby, CEO of BT, highlighted how the company is playing a key role in developing the infrastructure that acts as the 'UK's real digital backbone'.
------
Título: Super Micro Computer (SMCI) Debuts the Industry’s Broadest AI Solution Stack for NVIDIA Blackwell
Enlace: https://finance.yahoo.com/news/super-micro-computer-smci-debuts-031505879.html?.tsrc=rss
Resumen: Super Micro Computer, Inc. (NASDAQ:SMCI) is one of the 15 AI Stocks Making Waves on Wall Street. On June 11, the company announced an expansion of the industry’s broadest portfolio of solutions designed for NVIDIA Blackwell Architecture to the European market. The 30+ solutions offer the most comprehensive and efficient solution stack for NVIDIA HGX B200, GB200 […]
------
Título: CrowdStrike (CRWD) Boosts AI Security With NVIDIA—Her

{'summary': "Allison Kirkby, CEO of BT, highlighted how the company is playing a key role in developing the infrastructure that acts as the 'UK's real digital backbone'.",
 'summary_detail': {'type': 'text/html',
  'language': None,
  'base': 'https://feeds.finance.yahoo.com/rss/2.0/headline?s=NVDA&region=US&lang=en-US',
  'value': "Allison Kirkby, CEO of BT, highlighted how the company is playing a key role in developing the infrastructure that acts as the 'UK's real digital backbone'."},
 'id': '0cdfdba7-aae7-4286-b236-5923a87c4ad5',
 'guidislink': False,
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://uk.finance.yahoo.com/news/bt-ceo-london-tech-week-uk-infrastructure-050059140.html?.tsrc=rss'}],
 'link': 'https://uk.finance.yahoo.com/news/bt-ceo-london-tech-week-uk-infrastructure-050059140.html?.tsrc=rss',
 'published': 'Fri, 13 Jun 2025 05:00:59 +0000',
 'published_parsed': time.struct_time(tm_year=2025, tm_mon=6, tm_mday=13, tm_hour=5, tm_min=0, tm_sec

In [122]:
from bs4 import BeautifulSoup
from bs4.element import Tag
import requests
import pprint
import json
import datetime as dt

In [137]:
# Año mínimo en el que hay noticias es 2009
# Las categorías disponibles son "press_release" y "blog_post"
min_year = 2009
categories = {
    "press_release": 21926,
    "blog_post": 21924}


year = 2023
category = categories["press_release"]

BASE_URL = "https://nvidianews.nvidia.com"
HEADERS={"User-Agent": "Mozilla/5.0"}

url = BASE_URL+"/news?q=&year={year}&c={category}".format(
    year=year, category=category, page=1)

In [ ]:
lista = []
response = requests.get(url, headers=HEADERS)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")

    pages = get_page_numbers(soup)
    n_pages = len(pages)
    print("Número de páginas:", pages)
    
    for page in pages:
        print(f"Obteniendo página: {page} de {n_pages}")
        page_url = f"{url}&page={page}"
        
        try:
            response = requests.get(page_url, headers={"User-Agent": "Mozilla/5.0"})
        
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, "html.parser")
                articles = soup.find_all("article")
                n_articles = len(articles)

                for id, article in enumerate(articles, start=1):
                    print(f"Obteniendo artículo {id} de {n_articles}")
                    c_article = get_article_info(article)
                    
                    response = requests.get(c_article["link"], headers={"User-Agent": "Mozilla/5.0"})
                    soup_article = BeautifulSoup(response.content, "html.parser")
                    
                    c_article["content"] = get_content_page(soup_article)
                    lista.append(c_article)
        except requests.RequestException as e:
            print(f"Error al obtener la página {page}: {e}")



# result = requests.get("https://finance.yahoo.com/quote/NVDA?p=NVDA&.tsrc=fin-srch", headers={"User-Agent": "Mozilla/5.0"})

Número de páginas: ['1', '2', '3', '4', '5', '6']


In [158]:
lista

[{'title': 'NVIDIA Brings Business Intelligence to Chatbots, Copilots and Summarization Tools With Enterprise-Grade Generative AI Microservice',
  'link': 'https://nvidianews.nvidia.com/news/nemo-retriever-generative-ai-microservice',
  'publish_date': '2023-11-28',
  'summary': 'AWS re:Invent—NVIDIA today announced a generative AI microservice that lets enterprises connect custom large language models to enterprise data to deliver highly accurate responses for their AI applications.',
  'content': 'AWS re:Invent—NVIDIA today announced a generative AI microservice that lets enterprises connect custom large language models to enterprise data to deliver highly accurate responses for their AI applications.\nNVIDIA NeMo™ Retriever— a new offering in theNVIDIA NeMofamily of frameworks and tools for building, customizing and deploying generative AI models — helps organizations enhance their generative AI applications with enterprise-graderetrieval-augmented generation(RAG) capabilities.\nAs 

In [38]:
page = 1
url_with_page = url+"&page={page}".format(page=page)
url_with_page

response = requests.get(url_with_page, headers={"User-Agent": "Mozilla/5.0"})

In [89]:
# Encuentra las páginas de navegación
# y extrae los números de página
def get_page_numbers(soup: BeautifulSoup) -> list:
    """Extracts page numbers from the pagination section of the soup object.
    This function looks for a div with class "paging" and retrieves all links
    that contain the text "for page" in their title attribute. It returns a list
    of page numbers as strings.
    
    Note: This function assumes that the pagination links are structured in a specific way, such as
        for the NVIDIA News website, where the links contain the text "for page" in their title attribute.

    Args:
        soup (BeautifulSoup): A BeautifulSoup object containing the parsed HTML content.

    Returns:
        list: A list of page numbers as strings extracted from the pagination links.
    """
    paging_links = soup.find("div", class_="paging").find_all('a')
    page_numbers = [a.text for a in paging_links if "for page" in a.get("title", "")]
    return page_numbers


In [93]:
articles = soup.find_all("article")

In [130]:
def get_article_info(article: Tag) -> dict:
    """Extracts information from an article Tag object.
    This function retrieves the title, link, publish_date, and summary from a given article Tag.
    It returns a dictionary containing these details.
    Args:
        article (Tag): A BeautifulSoup Tag object representing an article.
    Returns:
        dict: A dictionary containing the article's title, link, publish_date, and summary.
    Note: This function assumes that the article Tag has a specific structure, such as
        containing an <h3> tag for the title, an <a> tag for the link, a <span> tag for the date,
        and a <div> tag for the description.
    """

    title = article.find(
        "h3", class_="index-item-text-title").get_text(strip=True)
    link = BASE_URL + article.find("a")["href"]
    date = article.find(
        "span", class_="index-item-text-info-date").get_text(strip=True)
    date = dt.datetime.strptime(date, "%B %d, %Y").strftime('%Y-%m-%d')
    summary = article.find(
        "div", class_="index-item-text-description").get_text(strip=True)

    return {"title": title, "link": link, "publish_date": date, "summary": summary}

In [140]:
c_article = get_article_info(articles[0])
link = c_article["link"]

In [142]:
response = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})

soup_article = BeautifulSoup(response.content, "html.parser")

In [155]:
def get_content_page(soup: BeautifulSoup) -> str:
    """Extracts the main content of a page from a BeautifulSoup object.
    This function retrieves the text content from a div with class "article-body".
    It returns the text as a string, with leading and trailing whitespace removed.

    Args:
        soup (BeautifulSoup): A BeautifulSoup object containing the parsed HTML content.

    Returns:
        str: The text content of the article body, stripped of leading and trailing whitespace.
    """
    try: 
        paragraphs = soup.find("div", class_="article-body").find_all("p")
        texto = [paragraphs[p].get_text(strip=True) for p in  range(len(paragraphs))]
        return "\n".join(texto)
    except AttributeError:
        return "No content available"


In [148]:
paragraphs[0].get_text(strip=True)

'AWS re:Invent—NVIDIA today announced a generative AI microservice that lets enterprises connect custom large language models to enterprise data to deliver highly accurate responses for their AI applications.'

In [ ]:


c_article

{'title': 'NVIDIA Brings Business Intelligence to Chatbots, Copilots and Summarization Tools With Enterprise-Grade Generative AI Microservice',
 'link': 'https://nvidianews.nvidia.com/news/nemo-retriever-generative-ai-microservice',
 'publish_date': '2023-11-28',
 'summary': 'AWS re:Invent—NVIDIA today announced a generative AI microservice that lets enterprises connect custom large language models to enterprise data to deliver highly accurate responses for their AI applications.',
 'text': 'AWS re:Invent—NVIDIA today announced a generative AI microservice that lets enterprises connect custom large language models to enterprise data to deliver highly accurate responses for their AI applications.\nNVIDIA NeMo™ Retriever— a new offering in theNVIDIA NeMofamily of frameworks and tools for building, customizing and deploying generative AI models — helps organizations enhance their generative AI applications with enterprise-graderetrieval-augmented generation(RAG) capabilities.\nAs a semant

In [110]:
date_str = art.find("span", class_="index-item-text-info-date").get_text(strip=True)

In [117]:
art.find("a")["href"]

'/news/infosys-and-nvidia'